**[CVPR 2020] 3D Photography using Context-aware Layered Depth Inpainting**

[project website](https://shihmengli.github.io/3D-Photo-Inpainting/)

In [ ]:
#@title Check GPU
#@markdown - Tier List: (K80 < T4 < P100 < V100 < A100)
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c7386f4e-3762-429d-e9b0-c73c8930ed7f)


In [ ]:
# @title Optional RAM check

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


- **Prepare environment**


In [ ]:
!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install opencv-python==4.2.0.32
!pip3 install vispy==0.6.4
!pip3 install moviepy==1.0.2
!pip3 install transforms3d==0.3.1
!pip3 install networkx==2.3
!sudo apt install sed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sed is already the newest version (4.4-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgr

- **Download script and pretrained model**


In [ ]:
%cd /content/
!git clone https://github.com/vt-vl-lab/3d-photo-inpainting.git
%cd 3d-photo-inpainting
!sh download.sh

/content
fatal: destination path '3d-photo-inpainting' already exists and is not an empty directory.
/content/3d-photo-inpainting
mkdir: cannot create directory ‘checkpoints’: File exists
downloading from filebox ...
--2022-09-22 01:12:21--  https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/color-model.pth
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206331633 (197M)
Saving to: ‘color-model.pth’

color-model.pth     100%[===================>] 196.77M  14.4MB/s    in 37s     

2022-09-22 01:12:59 (5.29 MB/s) - ‘color-model.pth’ saved [206331633/206331633]

--2022-09-22 01:12:59--  https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/depth-model.pth
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request 

- **Switch off off-screen rendering**

In [ ]:
!sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

- **Please upload `.jpg` files to `/content/3d-photo-inpainting-master/image/`**
  - You can run this step multiple times to upload multiple `.jpg` files.

In [ ]:
#@title Delete example image and videos.
!rm /content/3d-photo-inpainting/image/moon.jpg
!rm /content/3d-photo-inpainting/video/moon_circle.mp4
!rm /content/3d-photo-inpainting/video/moon_dolly-zoom-in.mp4
!rm /content/3d-photo-inpainting/video/moon_swing.mp4
!rm /content/3d-photo-inpainting/video/moon_zoom-in.mp4

#Find a way to delete all four videos with one command.

rm: cannot remove '/content/3d-photo-inpainting/image/moon.jpg': No such file or directory
rm: cannot remove '/content/3d-photo-inpainting/video/moon_circle.mp4': No such file or directory
rm: cannot remove '/content/3d-photo-inpainting/video/moon_dolly-zoom-in.mp4': No such file or directory
rm: cannot remove '/content/3d-photo-inpainting/video/moon_swing.mp4': No such file or directory
rm: cannot remove '/content/3d-photo-inpainting/video/moon_zoom-in.mp4': No such file or directory


In [ ]:
#@title Upload files

%cd image
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
%cd ..

/content/3d-photo-inpainting/image


Saving batman_solo-final-scale-4_00x.jpg to batman_solo-final-scale-4_00x.jpg
Saving cleanplate-final-scale-4_00x.jpg to cleanplate-final-scale-4_00x.jpg
Saving matte-final-scale-4_00x.jpg to matte-final-scale-4_00x.jpg
User uploaded file "batman_solo-final-scale-4_00x.jpg" with length 6694530 bytes
User uploaded file "cleanplate-final-scale-4_00x.jpg" with length 29153785 bytes
User uploaded file "matte-final-scale-4_00x.jpg" with length 29834110 bytes
/content/3d-photo-inpainting


- **Execute the 3D Photo Inpainting**
  - Note: The 3D photo generation process usually takes about 2-3 minutes or more depending on the available computing resources.

In [ ]:
#@title (Optional) Copy images from Google Drive
#You'll first need to upload all images you want to process to /image on your Google Drive.

# 1. Mount your Google Drive.
#from google.colab import drive
#drive.mount('/content/drive')

# 2. Move images over from Google Drive.
#!cp /content/drive/MyDrive/image/ -r /content/3d-photo-inpainting/

In [ ]:
#@title Do the execution . . .
!python main.py --config argument.yml

Traceback (most recent call last):
  File "main.py", line 29, in <module>
    config = yaml.load(open(args.config, 'r'))
TypeError: load() missing 1 required positional argument: 'Loader'


In [ ]:
#@title Compress results.

!zip -r /content/3d-photo-inpainting/inpainting-videos.zip /content/3d-photo-inpainting/video/
!mv /content/3d-photo-inpainting/video /content/3d-photo-inpainting/trash


zip error: Nothing to do! (try: zip -r /content/3d-photo-inpainting/inpainting-videos.zip . -i /content/3d-photo-inpainting/video/)
mv: cannot stat '/content/3d-photo-inpainting/video': No such file or directory


- **The results are stored in the following directories**
  - Corresponding depth map estimated by [MiDaS](https://github.com/intel-isl/MiDaS.git) 
      - E.g. ```/content/3d-photo-inpainting/depth/moon.npy```
  - Inpainted 3D mesh
      - E.g. ```/content/3d-photo-inpainting/mesh/moon.ply```
  - Rendered videos with zoom-in motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_zoom-in.mp4```
  - Rendered videos with swing motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_swing.mp4```
  - Rendered videos with circle motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_circle.mp4```
  
    ![](https://drive.google.com/uc?id=1xBiZAeGfhsoAsG08N5UM8-f993p0G5HF)